In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('D:\\Dropbox\\stempy')
sys.path.append('D:\\Dropbox\\stemplot')
    
from stemplot import *

from stempy.io import *
from stempy.clustering import *
from stempy.plot import *
from stempy.denoise import *
from stempy.utils import *
from stempy.feature import *
from stempy.spatial import *
from stempy.manifold import *

## Find points from HAADF

In [3]:
dp = r'D:\Dropbox\data\Leyi\Dopant Valency Project\CoWSe2_total 9 datasets\CoWSe2_20211124'+'\\'

num = 27

file_name1 = dp + 'ADF1_{}.dm4'.format(num)

# load image
img = load_image(file_name1)

img = normalize_image(img, 0, 1)

In [4]:
img1 = remove_bg(img, 11)

imshow(img1)

In [5]:
get_window_size(img)

15

## denoising

In [6]:
n_components = 32
patch_size = 32
extraction_step = 4
imgf = denoise_svd(img1, n_components, patch_size, extraction_step) 

Extracting reference patches...
done in 0.19s.
Singular value decomposition...
done in 2.03s.
Reconstructing...
done in 1.12s.


In [7]:
imshow(imgf)

In [8]:
threshold = 0.3
pts = local_max(imgf, min_distance=1, threshold=threshold)

## Clustering using MAADF

In [9]:
file_name2 = dp + 'ADF2_{}.dm4'.format(num)

# load image
img = load_image(file_name2)

img = normalize_image(img, 0, 1)

img2 = remove_bg(img, 11)

In [10]:
imshow(img2)

In [11]:
size = 26
kp = KeyPoints(pts, img2, size)
ps = kp.extract_patches(size) 

In [12]:
imshow(ps)

In [13]:
zps = ZPs(n_max=10, size=ps.shape[1])
zps.fit(ps)
X = zps.moments
X_rot = X.rotinv() 

In [14]:
plot_pca(X, 2)

In [21]:
fg = ForceGraph8(X = X,
n_neighbors=10,
init_mode = 'pca',
num_iterations = 100,
num_negative_samples=10,
force_params1=(0, 2, 1, 1),
force_params2=(2, 5, 5, 1),
divide=0.5)

from time import time
t0 = time()
xy = fg.fit_transform(X)

Construct graph from data...
Initialize 2-d embedding using PCA layout...


In [24]:
lbs = seg_lbs(xy)

In [26]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 7.2))
ax.imshow(img, cmap='gray')
ax.scatter(kp.pts[:, 0], kp.pts[:, 1], color=colors_from_lbs(lbs)) 

In [30]:
fig, ax = plt.subplots(1, 1, figsize=(7.2, 7.2))
ax.scatter(xy[:, 0], xy[:, 1], color=colors_from_lbs(lbs), s=4)
ax.axis('equal') 

(-72.18617016015472, 73.00159317458859, -17.68136293135505, 21.07749119415184)

In [22]:
l= interactive_clusters(xy, img, kp.pts, ps, clip=False, s=1) 

In [17]:
len(l.ind), l.ind

(78,
 array([   9,   40,  160,  163,  336,  349,  384,  406,  684,  957, 1009,
        1011, 1042, 1170, 1290, 1423, 1467, 1633, 1678, 1724, 1772, 1783,
        2077, 2116, 2456, 2514, 2678, 2847, 2850, 3197, 3241, 3286, 3318,
        3362, 3413, 3493, 3755, 3757, 3808, 3878, 3885, 4008, 4221, 4275,
        4332, 4624, 4794, 4835, 5127, 5195, 5264, 5350, 5402, 5620, 5653,
        5664, 5674, 5781, 5827, 5992, 6102, 6179, 6211, 6269, 6442, 6517,
        6525, 6599, 6650, 6863, 6989, 7079, 7643, 8109, 8150, 8288, 8375,
        8423], dtype=int64))

In [16]:
kp.pts

array([[ 48,   8],
       [ 78,   8],
       [121,   8],
       ...,
       [452, 504],
       [461, 504],
       [482, 504]], dtype=int64)

In [17]:
np.save('pts_Ti_25.npy', kp.pts)

In [18]:
np.save('ind_Ti_25.npy', l.ind)